## [--- Day 7: Laboratories ---](https://adventofcode.com/2025/day/7)

In [ ]:
# Helpers for 2D vector math and grid representation
class vector2(tuple[int, int]):
    __slots__ = ()  # Prevent '__dict__' creation

    """Crude 2D vector wrapper for tuples"""

    def __new__(cls, x: int = 0, y: int = 0) -> "vector2":
        return super(vector2, cls).__new__(cls, (x, y))

    @property
    def x(self):
        return self[0]

    @property
    def y(self):
        return self[1]

    def rotated_left(self) -> "vector2":
        return vector2(self[1], -self[0])

    def rotated_right(self) -> "vector2":
        return vector2(-self[1], self[0])

    def reflected_around(self, origin: "vector2") -> "vector2":
        return vector2(-self[0] + 2 * origin[0], -self[1] + 2 * origin[1])

    def distance_to(self, other: "vector2") -> int:
        return abs(self[0] - other[0]) + abs(self[1] - other[1])

    def __add__(self, other) -> "vector2":
        return vector2(self[0] + other[0], self[1] + other[1])

    def __sub__(self, other) -> "vector2":
        return vector2(self[0] - other[0], self[1] - other[1])

    def __repr__(self) -> str:
        return f"({self[0]}, {self[1]})"


vector2.zero = vector2(0, 0)
vector2.up = vector2(0, -1)
vector2.down = vector2(0, 1)
vector2.left = vector2(-1, 0)
vector2.right = vector2(1, 0)


class Grid:
    def __init__(self, data: list[list[str]]) -> None:
        self.data = data
        self.width = len(data[0])
        self.height = len(data)

    def in_bounds(self, pos: vector2) -> bool:
        return 0 <= pos.x < self.width and 0 <= pos.y < self.height

    def __getitem__(self, pos: vector2 | tuple[int, int]) -> str:
        if self.in_bounds(pos):
            return self.data[pos[1]][pos[0]]

    def __setitem__(self, pos: vector2 | tuple[int, int], value: str) -> None:
        if self.in_bounds(pos):
            self.data[pos[1]][pos[0]] = value

    def __repr__(self) -> str:
        representation: str = ""
        next_element: str = ""
        for row in self.data:
            for cell in row:
                if cell == 0:
                    next_element = "."
                elif cell == "^":
                    next_element = "^"
                elif cell == "S":
                    next_element = "S"
                else:
                    next_element = str(cell)
                representation += next_element.rjust(3)
            representation += "\n"

        return representation

In [ ]:
from collections import deque
import numbers


def solve() -> None:
    UP, DOWN, LEFT, RIGHT = vector2.up, vector2.down, vector2.left, vector2.right
    with open("..\\data\\07.txt") as file:
        data = [list(line.strip()) for line in file.readlines()]

    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] == ".":
                data[i][j] = 0

    grid: Grid = Grid(data)
    START: vector2 = vector2(data[0].index("S"), 0)
    pos: vector2 = START + DOWN
    beams: deque[vector2] = deque([pos])
    splits: int = 0
    grid[START] = 1

    while beams:
        pos = beams.popleft()
        current_timelines = grid[pos + UP]
        cell = grid[pos]
        if isinstance(cell, numbers.Number):
            if cell == 0:
                beams.append(pos + DOWN)
            grid[pos] += current_timelines
        elif cell == "^":
            splits += 1
            left_split = pos + LEFT
            if grid[left_split] == 0:
                beams.append(left_split + DOWN)

            right_split = pos + RIGHT
            if grid[right_split] == 0:
                beams.append(right_split + DOWN)

            grid[left_split] += current_timelines
            grid[right_split] += current_timelines

        # print(grid)
        # print()

    final_timelines = grid.data[grid.height - 1]
    print(f"Splits: {splits}\nTotal timelines: {sum(final_timelines)}")


solve()

Splits: 1640
Total timelines: 40999072541589
